<a href="https://colab.research.google.com/github/DarioRugg/Self-Supervised_Pearson_Search/blob/pretrained_test/main_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setup

### Path

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd /content/drive/MyDrive/PoE_Project/Self-Supervised_Pearson_Search/self_supervision/JigsawPuzzlePytorch/

/content/drive/.shortcut-targets-by-id/1fmgF9LrJJENY7Hz3KuriiSG0gKv202-0/Code/Self-Supervised_Pearson_Search/self_supervision/JigsawPuzzlePytorch


# Actual code

## Running the model

In [ ]:
!python JigsawTrain.py ../../data/PWR_self_supervised/ --pretrained --gpu 0 --batch 64 --epochs 30

Using GPU 0
Process number: 957
Images: train 4563, validation 1141
Start training: lr 0.001000, batch size 64, classes 1000
Checkpoint: checkpoints/
Learning Rate 0.001000
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
[ 1/30]     0) [batch load  17.524sec, net 0.39sec], LR 0.00100, Loss:  6.900, Accuracy  0.00%
[ 1/30]    20) [batch load  16.831sec, net 0.26sec], LR 0.00100, Loss:  6.905, Accuracy  0.00%
[ 1/30]    40) [batch load  16.905sec, net 0.26sec], LR 0.00100, Loss:  6.909, Accuracy  0.00%
[ 1/30]    60) [batch load  16.809sec, net 0.26sec], LR 0.00100, Loss:  6.897, Accuracy  0.00%
Learning Rate 0.001000
[ 2/30]    80) [batc

In [ ]:
!python JigsawTrain.py ../../data/PWR_self_supervised/ --not-pretrained --gpu 0 --batch 64 --epochs 3

Using GPU 0
Process number: 344
Images: train 4563, validation 1141
Start training: lr 0.001000, batch size 64, classes 1000
Checkpoint: checkpoints/
Learning Rate 0.001000
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
[ 1/ 3]     0) [batch load  20.318sec, net 0.13sec], LR 0.00100, Loss:  6.907, Accuracy  0.00%
Traceback (most recent call last):
  File "JigsawTrain.py", line 201, in <module>
    main()
  File "JigsawTrain.py", line 124, in main
    for i, (images, labels, original) in enumerate(train_loader):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 521, in __next__
    data = self._next_da

Tests with resnet

In [ ]:
import torch
import torch.nn as nn
from torch import cat
import torch.nn.init as init
import torchvision.models as models

In [ ]:
res50_model = models.resnet50(pretrained=True)  

In [ ]:
feature_extractor = nn.Sequential(*list(res50_model.children())[:-1])
pretrained_last_layers = nn.Sequential(*list(res50_model.children())[-1:])

In [ ]:
list(feature_extractor.children())

[Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False),
 BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True),
 ReLU(inplace=True),
 MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False),
 Sequential(
   (0): Bottleneck(
     (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
     (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
     (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
     (relu): ReLU(inplace=True)
     (downsample): Sequential(
       (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
       (1): BatchNorm2d(256, eps=1e-05, momentum

In [ ]:
list(pretrained_last_layers.children())

[Linear(in_features=2048, out_features=1000, bias=True)]

In [ ]:
class Network(nn.Module):

    def __init__(self, classes=1000):
        super(Network, self).__init__()

        res50_model = models.resnet50(pretrained=True)        
        self.feature_extractor = nn.Sequential(*list(res50_model.children())[:-1])
        self.pretrained_last_layers = nn.Sequential(*list(res50_model.children())[-1:])

        self.classifier = nn.Sequential()
        self.classifier.add_module('final_fc',nn.Linear(2048*9, classes))

    def save(self,checkpoint):
        res50_model_finetuned = nn.Sequential([self.feature_extractor, self.pretrained_last_layers])
        torch.save(res50_model_finetuned.state_dict(), checkpoint)
    
    def forward(self, x):
        B,T,C,H,W = x.size()
        x = x.transpose(0,1)

        x_list = []
        for i in range(9):
            z = self.feature_extractor(x[i])
            z = z.view([B,1,-1])
            x_list.append(z)

        x = cat(x_list,1)
        x = self.fc7(x.view(B,-1))
        x = self.classifier(x)

        return x


def weights_init(model):
    if type(model) in [nn.Conv2d,nn.Linear]:
        nn.init.xavier_normal(model.weight.data)
        nn.init.constant(model.bias.data, 0.1)
    